In [1]:
import pandas as pd
import numpy as np

# USING IMPORT EMAILS TO PARSE THORUGH MESSAGES
import email
from email.parser import Parser

import unicodedata
import re
import json
import os

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
from time import strftime
#from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Intensity Score (sentiment score)
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Polarity / Subjectivity scores
from textblob import TextBlob

In [3]:
df = pd.read_csv('01-clean_data.csv', index_col=False)

df.head()

,Unnamed: 0,file,subject,sender,date,content,clean,tokenize,stop_words,lemmatize,intensity,polarity,subjectivity,is_poi,is_internal
0,0,allen-p/_sent_mail/1.,NaN,phillip.allen@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n,here is our forecast,here is our forecast,forecast,forecast,0.0000,0.00,0.000000,False,True
1,1,allen-p/_sent_mail/10.,Re:,phillip.allen@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,traveling business meeting takes fun trip espe...,traveling business meeting take fun trip espec...,0.9313,0.15,0.613333,False,True
2,2,allen-p/_sent_mail/100.,Re: test,phillip.allen@enron.com,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!,test successful way to go,test successful way to go,test successful way go,test successful way go,0.5859,0.75,0.950000,False,True
3,3,allen-p/_sent_mail/1000.,NaN,phillip.allen@enron.com,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy,\n\n Can you send me a schedule of the s...",randy can you send me a schedule of the sala...,randy can you send me a schedule of the salary...,randy send schedule salary level everyone sche...,randy send schedule salary level everyone sche...,0.0000,0.00,0.000000,False,True
4,4,allen-p/_sent_mail/1001.,Re: Hello,phillip.allen@enron.com,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.,let's shoot for tuesday at 1145,let ' s shoot for tuesday at 1145,let ' shoot tuesday 1145,let ' shoot tuesday 1145,-0.3400,0.00,0.000000,False,True


In [4]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

df.head(2)

,file,subject,sender,date,content,clean,tokenize,stop_words,lemmatize,intensity,polarity,subjectivity,is_poi,is_internal
0,allen-p/_sent_mail/1.,NaN,phillip.allen@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n,here is our forecast,here is our forecast,forecast,forecast,0.0000,0.00,0.000000,False,True
1,allen-p/_sent_mail/10.,Re:,phillip.allen@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,traveling business meeting takes fun trip espe...,traveling business meeting take fun trip espec...,0.9313,0.15,0.613333,False,True


In [5]:
df.to_csv('01-clean_data.csv', index = False)

In [6]:
df.shape

(517401, 14)

# you can see what is going with nulls here
- If content column is special character, clean function will make in a Nan
- if cleaned content is an empty string, tokenize function will make it a Nan
- if tokenized content only contains a stop word, lemmatized function will make it a Nan

In [7]:
df.head()

,file,subject,sender,date,content,clean,tokenize,stop_words,lemmatize,intensity,polarity,subjectivity,is_poi,is_internal
0,allen-p/_sent_mail/1.,NaN,phillip.allen@enron.com,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n,here is our forecast,here is our forecast,forecast,forecast,0.0000,0.00,0.000000,False,True
1,allen-p/_sent_mail/10.,Re:,phillip.allen@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,traveling business meeting takes fun trip espe...,traveling business meeting take fun trip espec...,0.9313,0.15,0.613333,False,True
2,allen-p/_sent_mail/100.,Re: test,phillip.allen@enron.com,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!,test successful way to go,test successful way to go,test successful way go,test successful way go,0.5859,0.75,0.950000,False,True
3,allen-p/_sent_mail/1000.,NaN,phillip.allen@enron.com,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy,\n\n Can you send me a schedule of the s...",randy can you send me a schedule of the sala...,randy can you send me a schedule of the salary...,randy send schedule salary level everyone sche...,randy send schedule salary level everyone sche...,0.0000,0.00,0.000000,False,True
4,allen-p/_sent_mail/1001.,Re: Hello,phillip.allen@enron.com,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.,let's shoot for tuesday at 1145,let ' s shoot for tuesday at 1145,let ' shoot tuesday 1145,let ' shoot tuesday 1145,-0.3400,0.00,0.000000,False,True


In [8]:
df.isna().sum()

file                0
subject         19187
sender              0
date                0
content             0
clean              45
tokenize           54
stop_words        287
lemmatize         287
intensity           0
polarity            0
subjectivity        0
is_poi              0
is_internal         0
dtype: int64

In [19]:
df[(df.stop_words.isna() == True) &(df.tokenize.isna() == False)]

,file,subject,sender,date,content,clean,tokenize,stop_words,lemmatize,intensity,polarity,subjectivity,is_poi,is_internal
160,allen-p/_sent_mail/239.,Re:,phillip.allen@enron.com,"Mon, 26 Jun 2000 07:13:00 -0700 (PDT)",no,no,no,NaN,NaN,0.0,0.0,0.0,False,True
753,allen-p/all_documents/238.,Re:,phillip.allen@enron.com,"Mon, 26 Jun 2000 07:13:00 -0700 (PDT)",no,no,no,NaN,NaN,0.0,0.0,0.0,False,True
2567,allen-p/sent/192.,Re:,phillip.allen@enron.com,"Mon, 26 Jun 2000 07:13:00 -0700 (PDT)",no,no,no,NaN,NaN,0.0,0.0,0.0,False,True
6457,arnold-j/sent_items/188.,NaN,john.arnold@enron.com,"Fri, 15 Jun 2001 07:40:16 -0700 (PDT)",here she is...\n,here she is,here she is,NaN,NaN,0.0,0.0,0.0,False,True
6817,arnold-j/sent_items/517.,NaN,john.arnold@enron.com,"Tue, 9 Oct 2001 08:33:52 -0700 (PDT)",are you there?,are you there,are you there,NaN,NaN,0.0,0.0,0.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508959,williams-j/sent_items/96.,NaN,trading <.williams@enron.com>,"Mon, 4 Jun 2001 09:56:37 -0700 (PDT)",Here it is:\n\n\n,here it is,here it is,NaN,NaN,0.0,0.0,0.0,False,True
516391,zipper-a/sent_items/108.,Cargill,andy.zipper@enron.com,"Fri, 1 Jun 2001 07:53:22 -0700 (PDT)",where are we at with them ?,where are we at with them,where are we at with them,NaN,NaN,0.0,0.0,0.0,False,True
516545,zipper-a/sent_items/249.,RE: ERcot vols,andy.zipper@enron.com,"Tue, 9 Oct 2001 05:50:48 -0700 (PDT)",where are they ?\n,where are they,where are they,NaN,NaN,0.0,0.0,0.0,False,True
516668,zipper-a/sent_items/44.,RE: a test message,andy.zipper@enron.com,"Fri, 28 Dec 2001 09:25:30 -0800 (PST)",who are you ?,who are you,who are you,NaN,NaN,0.0,0.0,0.0,False,True


In [15]:
print(df.tokenize[151719])

In [16]:
df.clean[151719]

'    '